In [1]:
from inspect import signature
from functools import wraps

def typeassert(*ty_args, **ty_kwargs):
    def decorate(func):
        # If in optimized mode, disable type checking
        if not __debug__:
            return func
        # Map function argument names to supplied types
        sig = signature(func)
        bound_types = sig.bind_partial(*ty_args, **ty_kwargs).arguments
        @wraps(func)
        def wrapper(*args, **kwargs):
            bound_values = sig.bind(*args, **kwargs)
            # Enforce type assertions across supplied arguments
            for name, value in bound_values.arguments.items():
                if name in bound_types:
                    if not isinstance(value, bound_types[name]):
                        raise TypeError(
                        'Argument {} must be {}'.format(name, bound_types[name])
                        )
            return func(*args, **kwargs)
        return wrapper
    return decorate

In [2]:
@typeassert(int,int)
def add(x, y):
    return x + y

In [3]:
add(2,3)

5

In [4]:
add(2,'hello')

TypeError: Argument y must be <class 'int'>

In [5]:
@typeassert(int, z=int)
def spam(x, y, z=42):
    print(x, y, z)

In [6]:
spam(1,2,3)

1 2 3


In [7]:
spam(1,'hello',3)

1 hello 3


In [8]:
spam(1,'hello','world')

TypeError: Argument z must be <class 'int'>

In [9]:
from inspect import signature

sig = signature(spam)

In [10]:
print(sig)

(x, y, z=42)


In [11]:
sig.parameters

mappingproxy({'x': <Parameter "x">,
              'y': <Parameter "y">,
              'z': <Parameter "z=42">})

In [12]:
sig.parameters['z'].name

'z'

In [13]:
sig.parameters['z'].default

42

In [14]:
sig.parameters['z'].kind

<_ParameterKind.POSITIONAL_OR_KEYWORD: 1>

In [15]:
bound_types = sig.bind_partial(int, z=int)

In [16]:
bound_types

<BoundArguments (x=<class 'int'>, z=<class 'int'>)>

In [17]:
bound_types.arguments

OrderedDict([('x', int), ('z', int)])

In [18]:
bound_values = sig.bind(1, 2, 3)
bound_values

<BoundArguments (x=1, y=2, z=3)>